In [4]:
#importing libraries
import os
import shutil
import glob
import random
import PIL
import time
import json
import cv2 as cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import matplotlib.image as img
from PIL import Image
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import img_to_array, ImageDataGenerator
import tensorflow as tf
from tensorflow.keras import datasets, layers, models







In [3]:
os.mkdir('train')    

What is Captcha (Completely Automated Public Turing test to Tell Computers and Humans Apart)???¶
According to Wikipedia, a CAPTCHA is a type of challenge–response test used in computing to determine whether or not the user is human.


How does a CAPTCHA work?
The idea is that a computer program such as a bot will be unable to interpret the distorted letters, while a human being, who is used to seeing and interpreting letters in all kinds of contexts – different fonts, different handwritings, etc. – will usually be able to identify them.

CAPTCHAs were designed to prevent computers from automatically filling out forms by verifying that you are a real person. But with the rise of deep learning and computer vision, they can now often be defeated easily.


In [5]:
DATA_DIR='../input/captcha-version-2-images/samples/*png'
H,W,C=200,50,1
N_LABELS=128
D=5
label=[]
image=[]
image_list=[f for f in glob.glob(DATA_DIR,recursive=True)]   
for i in image_list:
    l=str(i).split('/')[-1].replace('.png','')
    im=str(i)
    label.append(l)
    image.append(im)
df=pd.DataFrame(columns=['image','label'])  
df['image']=image
df['label']=label


In [6]:
#shape 
cv2.imread(df['image'][1],cv2.IMREAD_GRAYSCALE).shape

In [7]:


fig=plt.figure(figsize=(10.00,10.50))
#plt.rcParams["figure.autolayout"] = True
fig.suptitle(' Preprocessed images ', fontsize=50)
img1=cv2.imread(df['image'][1],cv2.IMREAD_GRAYSCALE)
img2=cv2.imread(df['image'][2],cv2.IMREAD_GRAYSCALE)
imga1 = cv2.adaptiveThreshold(img1, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 145, 0)
imga2=cv2.adaptiveThreshold(img2, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 145, 0)
kernel = np.ones((5,5),np.uint8)
imagew1 = cv2.morphologyEx(imga1, cv2.MORPH_CLOSE, kernel)
imagew2 = cv2.morphologyEx(imga2, cv2.MORPH_CLOSE, kernel)
kernel = np.ones((2,2),np.uint8)
imaged1= cv2.dilate(imagew1, kernel, iterations = 1)
imaged2 = cv2.dilate(imagew2, kernel, iterations = 1)
imagesharp1 = cv2.GaussianBlur(imaged1, (5,5), 0) 
imagesharp2 = cv2.GaussianBlur(imaged2, (5,5), 0) 
plt.subplot(10, 2, 1)
plt.title('Greyscaled image 1')
plt.imshow(img1)
plt.subplot(10, 2, 2)
plt.title('Greyscaled image 2')
plt.imshow(img2)
plt.subplot(10, 2, 3)
plt.title('Adaptive thresholding image 1')
plt.imshow(imga1)
plt.subplot(10, 2, 4)
plt.title('Adaptive thresholding image 2')
plt.imshow(imga2)
plt.subplot(10, 2, 5)
plt.title('MropholgyEx image 1')
plt.imshow(imagew1)
plt.subplot(10, 2, 6)
plt.title('MoropholgyEx image 2')
plt.imshow(imagew2)
plt.subplot(10, 2, 7)
plt.title('Dilated image 1')
plt.imshow(imaged1)
plt.subplot(10, 2, 8)
plt.title('Dilated image 2')
plt.imshow(imaged2)
plt.subplot(10, 2, 9)
plt.title('Gaussian blur image 1')
plt.imshow(imagesharp1, cmap="Blues_r")
plt.subplot(10, 2, 10)
plt.title('Gaussian blur image 2')
plt.imshow(imagesharp2, cmap="Accent_r")
plt.show() 


In [8]:
from tensorflow.keras.utils import to_categorical
from PIL import Image
images=[]
labels=[]

vocabulary = {'2','3','4','5','6','7','8','b','c','d','e','f','g','m','n','p','w','x','y'}
char_to_num = {'2':0,'3':1,'4':2,'5':3,'6':4,'7':5,'8':6,'b':7,'c':8,'d':9,'e':10,'f':11,'g':12,'m':13,'n':14,'p':15,'w':16,'x':17,'y':18}

for i in df['image']:
            
           
            im = cv2.imread(i,cv2.IMREAD_GRAYSCALE)
            im = cv2.rotate(im, cv2.ROTATE_90_CLOCKWISE)
            
            im = cv2.adaptiveThreshold(im, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 145, 0)
            kernel = np.ones((5,5),np.uint8)
            imagew = cv2.morphologyEx(im, cv2.MORPH_CLOSE, kernel)
            kernel = np.ones((2,2),np.uint8)
            imagew = cv2.dilate(imagew, kernel, iterations = 1)
            imagesharp = cv2.GaussianBlur(imagew, (5,5), 0)
        
            
            im=np.array(imagesharp) / 255.0
            images.append(im)
for i in df['label']:
      label = list(map(lambda x:char_to_num[x], i))
      labels.append(label)
     # l'abels.append([to_categorical(ord(i_n), N_LABELS) for i_n in i])
#converting to array for feeding to model        
X=np.array(images)
y=np.array(labels)       
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = 42)                    
x_train = np.expand_dims(X_train, -1)
x_test=np.expand_dims(X_test, -1)
            

In [9]:

from tensorflow import keras
from tensorflow.keras import layers
# Let's create a new CTCLayer by subclassing
#CTC layer creation
class CTCLayer(layers.Layer):
    def __init__(self, name=None):
        super().__init__(name=name)
        self.loss_fn = keras.backend.ctc_batch_cost

    def call(self, y_true, y_pred):
        # Compute the training-time loss value and add it to the layer using `self.add_loss()`.
        batch_len = tf.cast(tf.shape(y_true)[0], dtype="int64")
        input_length = tf.cast(tf.shape(y_pred)[1], dtype="int64")
        label_length = tf.cast(tf.shape(y_true)[1], dtype="int64")

        input_length = input_length * tf.ones(shape=(batch_len, 1), dtype="int64")
        label_length = label_length * tf.ones(shape=(batch_len, 1), dtype="int64")

        loss = self.loss_fn(y_true, y_pred, input_length, label_length)
        self.add_loss(loss)

        # At test time, just return the computed predictions
        return y_pred

def build_model():
    
    # Inputs to the model
    input_img = layers.Input(shape=(200,50,1), name="image", dtype="float32") 
    labels = layers.Input(name="label", shape=(None,), dtype="float32")

    # First conv block
    x = layers.Conv2D(32,(3, 3),activation="relu",kernel_initializer="he_normal",padding="same",name="Conv1")(input_img)
    x = layers.MaxPooling2D((2, 2), name="pool1")(x)

    # Second conv block
    x = layers.Conv2D(64,(3, 3),activation="relu",kernel_initializer="he_normal",padding="same",name="Conv2")(x)
    x = layers.MaxPooling2D((2, 2), name="pool2")(x)

    # We have used two max pool with pool size and strides 2.
    # Hence, downsampled feature maps are 4x smaller. The number of
    # filters in the last layer is 64. Reshape accordingly before
    # passing the output to the RNN part of the model 
    x = layers.Reshape(target_shape=(50, 768), name="reshape")(x)
  
    x = layers.Dense(64, activation="relu", name="dense1")(x)
    x = layers.Dropout(0.2)(x)

    # RNNs
    x = layers.Bidirectional(layers.LSTM(128, return_sequences=True, dropout=0.25))(x)
    x = layers.Bidirectional(layers.LSTM(64, return_sequences=True, dropout=0.25))(x)

    # Output layer
    x = layers.Dense(20, activation="softmax", name="dense2")(x) # 20 = 19 characters + UKN

    # Add CTC layer for calculating CTC loss at each step
    output = CTCLayer(name="ctc_loss")(labels, x)

    # Define the model
    model = keras.models.Model(inputs=[input_img, labels], outputs=output, name="captcha_cracking")
   
    # Compile the model and return
    model.compile(optimizer=keras.optimizers.Adam())
    return model


# Get the model
model = build_model()
model.summary()

In [10]:
es = keras.callbacks.EarlyStopping(monitor='val_loss',
                                   patience=5,
                                   restore_best_weights=True)

batch_size = 64
valid_batch_size = 64
model_np=model.fit([x_train,y_train],epochs=50, validation_data=[x_test, y_test],shuffle=True,callbacks=[es])


In [11]:
model.evaluate([x_test, y_test], batch_size=64)

In [12]:
# summarize history for loss
plt.plot(model_np.history['loss'])
plt.plot(model_np.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()  

In [13]:
model.save('captchamodel.h5')


In [ ]:
model = keras.models.load_model('.h5')

In [14]:

num_to_char = {'-1':'UKN','0':'2','1':'3','2':'4','3':'5','4':'6','5':'7','6':'8','7':'b','8':'c','9':'d','10':'e','11':'f','12':'g','13':'m','14':'n','15':'p','16':'w','17':'x','18':'y'}
prediction_model = keras.models.Model(
    model.get_layer(name="image").input, model.get_layer(name="dense2").output
)
for i in range(10):
        y_pred = prediction_model.predict(np.expand_dims(x_test[i], axis=0))
        y_pred = keras.backend.ctc_decode(y_pred , input_length=np.ones(1)*50, greedy=True) # decoding -> y_pred[0].shape = (104,5)
        y_pred = y_pred[0][0][0:104,0:5].numpy() 
        print(f"predicted :{str(list(map(lambda x:num_to_char[str(x)], y_pred[0])))},actual :{str(list(map(lambda x:num_to_char[str(x)], y_test[i])))} ")



### optimization

In [15]:

from tensorflow import keras
from tensorflow.keras import layers
# Let's create a new CTCLayer by subclassing
class CTCLayer(layers.Layer):
    def __init__(self, name=None):
        super().__init__(name=name)
        self.loss_fn = keras.backend.ctc_batch_cost

    def call(self, y_true, y_pred):
        # Compute the training-time loss value and add it to the layer using `self.add_loss()`.
        batch_len = tf.cast(tf.shape(y_true)[0], dtype="int64")
        input_length = tf.cast(tf.shape(y_pred)[1], dtype="int64")
        label_length = tf.cast(tf.shape(y_true)[1], dtype="int64")

        input_length = input_length * tf.ones(shape=(batch_len, 1), dtype="int64")
        label_length = label_length * tf.ones(shape=(batch_len, 1), dtype="int64")

        loss = self.loss_fn(y_true, y_pred, input_length, label_length)
        self.add_loss(loss)

        # At test time, just return the computed predictions
        return y_pred

def build_model(hp):
    
    # Inputs to the model
    input_img = layers.Input(shape=(200,50,1), name="image", dtype="float32") 
    labels = layers.Input(name="label", shape=(None,), dtype="float32")

    # First conv block
    x = layers.Conv2D(32,(3, 3),activation="relu",kernel_initializer="he_normal",padding="same",name="Conv1")(input_img)
    x = layers.MaxPooling2D((2, 2), name="pool1")(x)

    # Second conv block
    x = layers.Conv2D(64,(3, 3),activation="relu",kernel_initializer="he_normal",padding="same",name="Conv2")(x)
    x = layers.MaxPooling2D((2, 2), name="pool2")(x)

    # We have used two max pool with pool size and strides 2.
    # Hence, downsampled feature maps are 4x smaller. The number of
    # filters in the last layer is 64. Reshape accordingly before
    # passing the output to the RNN part of the model 
    x = layers.Reshape(target_shape=(50, 768), name="reshape")(x)
  
    x = layers.Dense(64, activation="relu", name="dense1")(x)
    x = layers.Dropout(0.2)(x)

    # RNNs
    x = layers.Bidirectional(layers.LSTM(128, return_sequences=True, dropout=0.25))(x)
    x = layers.Bidirectional(layers.LSTM(64, return_sequences=True, dropout=0.25))(x)

    # Output layer
    x = layers.Dense(20, activation="softmax", name="dense2")(x) # 20 = 19 characters + UKN

    # Add CTC layer for calculating CTC loss at each step
    output = CTCLayer(name="ctc_loss")(labels, x)

    # Define the model
    model = keras.models.Model(inputs=[input_img, labels], outputs=output, name="captcha_cracking")
    hp_l=hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    # Compile the model and return
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_l))
    return model


In [22]:
import keras_tuner as kt
tuner = kt.tuners.BayesianOptimization(
    build_model,
    objective='val_loss',max_trials=1,overwrite=True
    )

In [23]:
tuner.search([x_train,y_train],epochs=100, validation_data=[x_test, y_test])
best_model = tuner.get_best_models()[0]
best_hyperparameters = tuner.get_best_hyperparameters(1)[0]

In [24]:
model_op=tuner.hypermodel.build(best_hyperparameters)
model_op.summary()
model_p=model_op.fit([x_train,y_train],epochs=50, validation_data=[x_test, y_test],shuffle=True)

In [ ]:
# summarize history for loss
plt.plot(model_p.history['loss'])
plt.plot(model_p.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()  

In [ ]:
num_to_char = {'-1':'UKN','0':'2','1':'3','2':'4','3':'5','4':'6','5':'7','6':'8','7':'b','8':'c','9':'d','10':'e','11':'f','12':'g','13':'m','14':'n','15':'p','16':'w','17':'x','18':'y'}
prediction_model = keras.models.Model(
    model_op.get_layer(name="image").input, model_op.get_layer(name="dense2").output
)
prediction_model.summary()
for i in range(10):
    
    y_pred = prediction_model.predict(np.expand_dims(x_test[13], axis=0))
    print(y_pred.shape)
    y_pred = keras.backend.ctc_decode(y_pred , input_length=np.ones(1)*50, greedy=True) # decoding -> y_pred[0].shape = (104,5)
    y_pred = y_pred[0][0][0:104,0:5].numpy() 
    print(y_pred)
    print(f"predicted :{str(list(map(lambda x:num_to_char[str(x)], y_pred[0])))},actual :{str(list(map(lambda x:num_to_char[str(x)], y_test[13])))} ")

